In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve

from numba import njit
from time import time

import h5py
from datetime import datetime
import os

# Concepts
Since I am in a spherically symmetric spacetime, I can fix the angles to arbitrary values and forget them. So I only need to keep track of the radius. The 3D problem is effectively reduced to a 1D one!!! Radial differenciation is then reduced to simply the derivative along the array.

In [2]:
class Fields():
    
    def __init__(self, R = 10, N = 110):
        # Domain is a sphere of radius R
        self.Rmax = R
        self.N    = N
        
        self.dR  = R / N
    
        self.r = np.array([(j - 0.5)*self.dR for j in range(1,N+1)], dtype = np.float64)
        
        self.fields = np.zeros(7 * N)
        
        self.psi = 1 + 1 / 4 / self.r  #r_s = 1
    
    def A(self):
        return self.fields[0:self.N]
    def B(self):
        return self.fields[self.N:2*self.N]
    def DA(self):
        return self.fields[2*self.N:3*self.N]
    def DB(self):
        return self.fields[3*self.N:4*self.N]
    def KA(self):
        return self.fields[4*self.N:5*self.N]
    def KB(self):
        return self.fields[5*self.N:6*self.N]
    def al(self):
        return self.fields[6*self.N:7*self.N]
    
    def IC_1plusLogSlicing(self):
        """
        1+Log Slicing gauge condition.
                A = B = psi
                DA = DB = d/dr ln(x), x = A, B
                KA = KB = 0
                alpha   = 1
        """
        self.fields[0       :  self.N]  = np.copy(self.psi)
        self.fields[  self.N:2*self.N]  = np.copy(self.psi)
        for i in range(N):
            self.fields[2*N+i] = der_r(np.log(self.A()), i, self.dR)
            self.fields[3*N+i] = der_r(np.log(self.B()), i, self.dR)
        self.fields[6*self.N:7*self.N] = np.ones(self.N)
        

In [3]:

def der_r(f, i, dR):
    if i == 0:
        # Forward
        return 0.5 * (- 3 * f[0] + 4 * f[1] - f[2]) / dR
    elif i == len(f)-1:
        # Backward
        return 0.5 * (3 * f[-1] - 4 * f[-2] + f[-3]) / dR
    else:
        # Central
        return 0.5 * (f[i+1] - f[i-1]) / dR

def sec_der_r(f, i, dR):
    if i == 0:
        # Forward
        return (2 * f[0] - 5 * f[1] + 4 * f[2] - f[3]) / dR**3
    elif i == len(f)-1:
        # Backward
        return (2 * f[-1] -5 * f[-2] + 4 * f[-3] - f[-4]) / dR**3
    else:
        # Central
        return (f[i+1] - 2 * f[i] + f[i-1]) / dR**2

# RHS

In [13]:

def ev_A(f0, i, r, dR, N):
    """
    field :: class object containing all the fields (self in Fields class)
    i,j,k   :: position in the grid
    """
    al = f0[6*N:7*N]
    A  = f0[0  :  N]
    KA = f0[4*N:5*N]
    
    return - 2 * al[i] * A[i] * KA[i]


def ev_B(f0, i, r, dR, N):
    """
    field :: class object containing all the fields (self in Fields class)
    i,j,k   :: position in the grid
    """
    al = f0[6*N:7*N]
    B  = f0[  N:2*N]
    KB = f0[5*N:6*N]
    
    return - 2 * al[i] * B[i] * KB[i]


def ev_DA(f0, i, r, dR, N):
    """
    field :: class object containing all the fields (self in Fields class)
    i,j,k   :: position in the grid
    """
    al = f0[6*N:7*N]
    KA = f0[4*N:5*N]

    p1 = KA[i] * der_r(np.log(al), i, dR)
    p2 = der_r(KA, i, dR)
    return - 2 * al[i] * (p1 + p2)


def ev_DB(f0, i, r, dR, N):
    """
    field :: class object containing all the fields (self in Fields class)
    i,j,k   :: position in the grid
    """
    al = f0[6*N:7*N]
    KB = f0[5*N:6*N]
    
    p1 = KB[i] * der_r(np.log(al), i, dR)
    p2 = der_r(KB, i, dR)
    return - 2 * al[i] * (p1 + p2)


def ev_KA(f0, i, r, dR, N):
    """
    field :: class object containing all the fields (self in Fields class)
    i,j,k   :: position in the grid
    """
    
    A  = f0[0  :  N]
    B  = f0[  N:2*N]
    DA = f0[2*N:3*N]
    DB = f0[3*N:4*N]
    KA = f0[4*N:5*N]
    KB = f0[5*N:6*N]
    al = f0[6*N:7*N]
    
    r = r[i]
    Dal = [der_r(np.log(al), j, dR) for j in range(N)]
    
    p1 = der_r(Dal + DB, i, dR)
    p2 = Dal[i]**2 + 0.5 * (- Dal[i] * DA[i] + DB[i]**2 - DA[i] * DB[i])
    p3 = - A[i] * KA[i] * (KA[i] + 2*KB[i])
    p4 = - (DA[i] - 2 * DB[i]) / r
    return - al[i] * (p1 + p2 + p3 + p4) / A[i]


def ev_KB(f0, i, r, dR, N):
    """
    field :: class object containing all the fields (self in Fields class)
    i,j,k   :: position in the grid
    """
    A  = f0[0  :  N]
    B  = f0[  N:2*N]
    DA = f0[2*N:3*N]
    DB = f0[3*N:4*N]
    KA = f0[4*N:5*N]
    KB = f0[5*N:6*N]
    al = f0[6*N:7*N]
    
    r = r[i]
    
    p1 = der_r(DB, i, dR)
    p2 = der_r(np.log(al), i, dR) * DB[i] + DB[i]**2 - 0.5 * DA[i] * DB[i]
    p3 = - (DA[i] - 2 * der_r(np.log(al), i, dR) - 4 * DB[i]) / r
    p4 = - 2 * (A[i] - B[i]) / (B[i] * r**2)
    p6 = al[i] * KB[i] * (KA[i] + 2 * KB[i])
    return - 0.5 * al[i] * (p1 + p2 + p3 + p4) / A[i] + p6


def ev_al(f0, i, r, dR, N):
    KA = f0[4*N:5*N]
    KB = f0[5*N:6*N]
    al = f0[6*N:7*N]
    
    return -2 * al[i] * (KA[i] + KB[i]*2)   ## Prvare KA - 2*KB

# RK4

In [14]:

def extrapolate(x_2, y_2, x_1, y_1, x_ex):
    return y_2 + (y_1 - y_2) * (x_ex - x_2) / (x_1 - x_2)

def eulerStep(fields, dt, k, fac):
    fields_dict = {
            0: fields.A,  1: fields.B,
            2: fields.DA, 3: fields.DB,
            4: fields.KA, 5: fields.KB,
            6: fields.al
                    }
    
    results = np.zeros(7 * fields.N)
    for f in range(7):
        init = f*fields.N
        fin  = (f+1)*fields.N
        results[init]  = fields_dict[f]()[0]
        results[init+1:fin] = fields_dict[f]()[1:] + k[init+1:fin] * fac * dt
        #results[fin-1] = extrapolate(fields.r[-3], results[fin - 3], fields.r[-2], results[fin - 2], fields.r[-1])
        return results

def RHS(f0, r, dr, N):
    func_dict = {
            0: ev_A,  1: ev_B,
            2: ev_DA, 3: ev_DB,
            4: ev_KA, 5: ev_KB,
            6: ev_al
            }
    rhs = np.zeros(7 * N)
    for f in range(7):
        for i in range(N):
            rhs[N * f + i] = func_dict[f](f0, i, r, dr, N)
    return rhs

def rk4(fields, dt):
    k1 = RHS(fields.fields, fields.r, fields.dR, fields.N)
    k2 = RHS(eulerStep(fields, dt, k1, 0.5), fields.r, fields.dR, fields.N)     # t0 + 0.5 * dt
    k3 = RHS(eulerStep(fields, dt, k2, 0.5), fields.r, fields.dR, fields.N)     # t0 + 0.5 * dt
    k4 = RHS(eulerStep(fields, dt, k3, 1  ), fields.r, fields.dR, fields.N)     # t0 +  dt
    return fields.fields + (k1/6 + k2/3 + k3/3 + k4/6) * dt / 2


# Apparent Horizon

In [15]:
def func(x, field):
    dB = np.zeros(field.N)
    for i in range(field.N):
        dB[i] = der_r(field.B, i, field.dR)
    tab = (2 / x + dB / field.B) / np.sqrt(field.A) - 2 * field.KB
    
    return f

def comp_appHorizon(field):
    root = fsolve(func, 1., args=(field,), full_output=True)
    return root

# Main

In [ ]:
SAVE = 0
dt = 0.0001
t = 0
tmax = 10
N = 100
R = 5

if SAVE:
    name = f'{datetime.now().strftime("%Y_%m_%d_%H_%M_%S")}_1pL_res.h5'
    h5f = h5py.File(name, 'w')
    sss = f'Each dataset contains the following variables:\n \
    \tA\n\
    \tB\n\
    \tD_A\n\
    \tD_B\n\
    \tK_A\n\
    \tK_B\n\
    \tD_alpha\n\
    \talpha\n\
    \nThe name of the dataset is the index of the time step (e.g. "i" is the first computation, at t = i*dt)\n\
    Parameters:\n\
    \tN = {N} (number of points in the radial direction)\n\
    \tdt = {dt} (timestep used)\n\
    \tt_max = {tmax} (number of time iterations)'
    h5f.attrs['Info']  = sss

fields = Fields(R = R, N = N)
fields.IC_1plusLogSlicing()

j = 0
t_tot = 0
while j < tmax:
    
    
    ti = time()
    fields.fields = np.copy(rk4(fields, dt))
    t_tot += time() - ti
    
    t += dt
    
    #dB = np.zeros(fields.N)
    #for i in range(fields.N):
    #    dB[i] = der_r(fields.B(), i, fields.dR)
    #    
    #tab = (2 / fields.r + dB / fields.B()) / np.sqrt(fields.A()) - 2 * fields.KB()
    #for i in range(len(tab)-1):
    #    if np.sign(tab[i]) != np.sign(tab[i+1]):
    #        print(f'Change at idx {i}-{i+1}: {tab[i]} {tab[i+1]}')
    
    #rh = comp_appHorizon(fields)
    #print(rh)
    
    if SAVE:
        reshaped_output = np.reshape(fields.fields, (7, fields.N))
        h5f.create_dataset(f'{j}', data=reshaped_output, compression = 9)
    j += 1

        
print(t_tot / tmax)        
if SAVE:    
    h5f.close()
    os.system(f'python3 Plots_1plusLog.py {name} {N} {tmax} {fields.dR}')